In [41]:
import pandas as pd
import os
import warnings
import datetime
from datetime import datetime
from dateutil.relativedelta import relativedelta

In [42]:
warnings.filterwarnings('ignore')

In [43]:
path_leave = r'C:\Users\charoenchai.chu\Desktop\Rules\finish\บางสะพาน_Model'
name_leave = 'ข้อมูลวันลา_3ปี'

file_path1 = os.path.join(path_leave, f'{name_leave}.xlsx')
df_leave = pd.read_excel(file_path1)
df_leave = df_leave[['โค้ด','ชือ - นามสกุล','ตำแหน่ง','วันที่เริ่มต้น','วันที่สิ้นสุด','รอบเวลา (1 วัน)']]
# df_leave.head()

In [44]:
#เปลี่ยน path
path_lone = r'C:\Users\charoenchai.chu\Desktop\Rules\finish\บางสะพาน_Model'
file_lone = 'รวมไฟล์_new'

file_path_lone = os.path.join(path_lone, file_lone + '.xlsx')
df_lone=pd.read_excel(file_path_lone , skiprows= 3)
df_lone['submitbranch']=df_lone['submitbranch'].astype(str).str.zfill(3)
# df_lone['journaltimestamp','recordtimestamp']
# df_lone.head()

In [45]:
# แปลงคอลัมน์ recordtimestamp และ journaltimestamp เป็น datetime
df_lone['recordtimestamp'] = pd.to_datetime(df_lone['recordtimestamp'], errors='coerce')
df_lone['journaltimestamp'] = pd.to_datetime(df_lone['journaltimestamp'], errors='coerce')

# ฟังก์ชันเพื่อเพิ่ม 1 ปีให้กับวันที่
def add_year(date):
    if pd.notna(date):  # ตรวจสอบว่าไม่เป็น NaT
        return date + relativedelta(years=1)
    return date

# เพิ่มวันที่ 1 ปี
df_lone['recordtimestamp'] = df_lone['recordtimestamp'].apply(add_year)
df_lone['journaltimestamp'] = df_lone['journaltimestamp'].apply(add_year)

# df_lone.head()

In [46]:
# join วันลา กับ lone
df_leave['โค้ด'] = df_leave['โค้ด'].astype(str).str.zfill(3)
df_lone['submitbranch'] = df_lone['submitbranch'].astype(str).str.zfill(3)

df_leave['วันที่เริ่มต้น'] = pd.to_datetime(df_leave['วันที่เริ่มต้น']).dt.date
df_leave['วันที่สิ้นสุด'] = pd.to_datetime(df_leave['วันที่สิ้นสุด']).dt.date
df_lone['date'] = pd.to_datetime(df_lone['recordtimestamp']).dt.date

# ฟิลเตอร์ข้อมูลในสาขาเดียวกันและวันที่ตรงกัน
data_df = df_lone.merge(df_leave, how='left', 
                           left_on=['date', 'submitbranch'], 
                           right_on=['วันที่เริ่มต้น', 'โค้ด'])
# data_df.to_excel(r'C:\Users\charoenchai.chu\Desktop\Rules\finish\บางสะพาน_Model\ทดสอบ.xlsx')

#### Set Rules

In [47]:
# มีการทำสัญญานอกเหนือจากเวลาทำงาน

# แปลงคอลัมน์ 'recordtimestamp' ให้เป็น datetime
df_lone['recordtimestamp'] = pd.to_datetime(df_lone['recordtimestamp'])

# กำหนดช่วงเวลาที่ต้องการกรอง
morning_start = pd.to_datetime('08:30:00').time()
morning_end = pd.to_datetime('12:00:00').time()
afternoon_start = pd.to_datetime('13:00:00').time()
afternoon_end = pd.to_datetime('17:00:00').time()

# ก่อน 08.30
before_morning_time = df_lone[(df_lone['recordtimestamp'].dt.time < morning_start)]

# # กรองข้อมูลที่ไม่อยู่ในช่วงเวลาทำงาน
# filtered_time = df_lone[~((df_lone['recordtimestamp'].dt.time >= morning_start) & (df_lone['recordtimestamp'].dt.time <= morning_end) |
#                     (df_lone['recordtimestamp'].dt.time >= afternoon_start) & (df_lone['recordtimestamp'].dt.time <= afternoon_end))]

# # แสดงข้อมูลที่กรองแล้ว
# # filtered_df['recordtimestamp']
before_morning_time['Rules01'] = 'มีการทำสัญญาก่อนเวลาทำงาน'
filtered_time1 = before_morning_time[['submitbranch','policyno','recordtimestamp','Rules01']]
filtered_time1

,submitbranch,policyno,recordtimestamp,Rules01
777,082,36932126,2022-06-10 08:29:41,มีการทำสัญญาก่อนเวลาทำงาน
2134,099,36422044,2023-05-03 08:19:43,มีการทำสัญญาก่อนเวลาทำงาน
2315,101,21927528,2023-04-05 08:29:58,มีการทำสัญญาก่อนเวลาทำงาน
4537,195,36045738,2022-10-05 08:17:50,มีการทำสัญญาก่อนเวลาทำงาน
4643,195,37366818,2022-06-25 08:18:31,มีการทำสัญญาก่อนเวลาทำงาน
...,...,...,...,...
24696,696,36845202,2024-04-17 08:22:35,มีการทำสัญญาก่อนเวลาทำงาน
25005,665,37047510,2023-04-25 08:27:20,มีการทำสัญญาก่อนเวลาทำงาน
25091,665,37101413,2023-03-23 08:25:00,มีการทำสัญญาก่อนเวลาทำงาน
25471,636,37327365,2023-11-23 08:12:45,มีการทำสัญญาก่อนเวลาทำงาน


In [48]:
# # พักเที่ยง ลันช์ไทม์
luch_time = df_lone[(df_lone['recordtimestamp'].dt.time > morning_end) & (df_lone['recordtimestamp'].dt.time < afternoon_start)]
# # หลัง 17.00
# after_afternoon_end = df_lone[(df_lone['recordtimestamp'].dt.time > afternoon_end)]

# # กรองข้อมูลที่ไม่อยู่ในช่วงเวลาที่กำหนด
# filtered_time = df_lone[~((df_lone['recordtimestamp'].dt.time >= morning_start) & (df_lone['recordtimestamp'].dt.time <= morning_end) |
#                     (df_lone['recordtimestamp'].dt.time >= afternoon_start) & (df_lone['recordtimestamp'].dt.time <= afternoon_end))]

# # แสดงข้อมูลที่กรองแล้ว
# # filtered_df['recordtimestamp']
luch_time['Rules02'] = 'มีการทำสัญญาช่วงเวลาพักเที่ยง'
filtered_time2 = luch_time[['submitbranch','policyno','recordtimestamp','Rules02']]
filtered_time2

,submitbranch,policyno,recordtimestamp,Rules02
1,032,90214035,2024-06-25 12:09:43,มีการทำสัญญาช่วงเวลาพักเที่ยง
8,032,32469316,2024-06-07 12:42:23,มีการทำสัญญาช่วงเวลาพักเที่ยง
40,032,35503919,2024-03-12 12:39:44,มีการทำสัญญาช่วงเวลาพักเที่ยง
43,032,22926184,2024-02-27 12:42:23,มีการทำสัญญาช่วงเวลาพักเที่ยง
48,032,90115416,2024-01-29 12:04:17,มีการทำสัญญาช่วงเวลาพักเที่ยง
...,...,...,...,...
26728,674,90221172,2023-04-07 12:50:47,มีการทำสัญญาช่วงเวลาพักเที่ยง
26738,672,90268525,2024-04-02 12:53:06,มีการทำสัญญาช่วงเวลาพักเที่ยง
26739,636,90271138,2024-03-18 12:41:32,มีการทำสัญญาช่วงเวลาพักเที่ยง
26740,850,90273073,2024-06-28 12:10:41,มีการทำสัญญาช่วงเวลาพักเที่ยง


In [49]:
# # หลัง 17.00
after_afternoon_end = df_lone[(df_lone['recordtimestamp'].dt.time > afternoon_end)]

# # แสดงข้อมูลที่กรองแล้ว
# # filtered_df['recordtimestamp']
after_afternoon_end['Rules03'] = 'มีการทำสัญญาหลังเวลาเลิกงาน'
filtered_time3 = after_afternoon_end[['submitbranch','policyno','recordtimestamp','Rules03']]
# filtered_time3

In [50]:
# มีพนักงานคนใดคนหนึ่งลาในช่วงเวลาทำงาน
# data_df1 = data_df[['โค้ด','policyno1']]
# ฟิลเตอร์ข้อมูลในสาขาเดียวกันและวันที่ตรงกัน
filter_leave = data_df[data_df['โค้ด'].notnull()]

filter_leave['Rules04'] = 'มีการสร้างสัญญาในวันที่อยู่สาขาคนเดียว'
filter_leave=filter_leave[['โค้ด','policyno','recordtimestamp','Rules04']]
filter_leave
# leave

,โค้ด,policyno,recordtimestamp,Rules04
18,032,38169239,2024-05-25 10:07:27,มีการสร้างสัญญาในวันที่อยู่สาขาคนเดียว
19,032,35588533,2024-05-23 14:34:18,มีการสร้างสัญญาในวันที่อยู่สาขาคนเดียว
26,032,37900822,2024-05-03 09:51:12,มีการสร้างสัญญาในวันที่อยู่สาขาคนเดียว
31,032,86017028,2024-04-19 11:12:46,มีการสร้างสัญญาในวันที่อยู่สาขาคนเดียว
33,032,90241056,2024-04-09 11:13:20,มีการสร้างสัญญาในวันที่อยู่สาขาคนเดียว
...,...,...,...,...
26712,850,90178243,2023-04-21 10:38:31,มีการสร้างสัญญาในวันที่อยู่สาขาคนเดียว
26725,634,90198171,2023-06-19 10:06:34,มีการสร้างสัญญาในวันที่อยู่สาขาคนเดียว
26730,691,90206047,2023-09-27 14:58:55,มีการสร้างสัญญาในวันที่อยู่สาขาคนเดียว
26739,634,90215564,2023-08-11 09:55:48,มีการสร้างสัญญาในวันที่อยู่สาขาคนเดียว


In [51]:
# ทำ left join โดยเริ่มจาก df1 กับ df2
df_left_join_1 = data_df.merge(filtered_time1, how='left', on=['submitbranch','policyno','recordtimestamp'])
df_left_join_2 = df_left_join_1.merge(filtered_time2, how='left', on=['submitbranch','policyno','recordtimestamp'])
df_left_join_3 = df_left_join_2.merge(filtered_time3, how='left', on=['submitbranch','policyno','recordtimestamp'])

# จากนั้นทำ left join กับ df3
df_left_join_4 = df_left_join_3.merge(filter_leave, how='left', on=['โค้ด','policyno','recordtimestamp'])

# แสดงข้อมูลที่ได้

In [52]:
df_left_join_4

,policyno,prename,firstname,lastname,policystatus1,policystatusdate1,policyno1,loandate,Policyloandate,registerno,...,โค้ด,ชือ - นามสกุล,ตำแหน่ง,วันที่เริ่มต้น,วันที่สิ้นสุด,รอบเวลา (1 วัน),Rules01,Rules02,Rules03,Rules04
0,35588533,000001,นิยม,เซี่ยงฉี,F,25710226,35588533,25670628,3558853325670628,35588533,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,90214035,000004,ภากร,สิงห์ลอ,I,25670625,90214035,25670625,9021403525670625,90214035,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,มีการทำสัญญาช่วงเวลาพักเที่ยง,NaN,NaN
2,37900855,000002,บานเย็น,ทิมทอง,C,25670623,37900855,25670620,3790085525670620,37900855,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,23224015,000003,นิษา,สมรูป,I,25650531,23224015,25670615,2322401525670615,23224015,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,36557624,000002,พรทิพย์,พุฒิลือชา,I,0,36557624,25670614,3655762425670614,36557624,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26787,90271138,000003,วิเชียร,มาพันธุ์,I,25670116,90271138,25670318,9027113825670318,90271138,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,มีการทำสัญญาช่วงเวลาพักเที่ยง,NaN,NaN
26788,90273073,000003,ปณิตา,กลัดเจ็ด,I,25670422,90273073,25670628,9027307325670628,90273073,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,มีการทำสัญญาช่วงเวลาพักเที่ยง,NaN,NaN
26789,90274255,000001,สุทัศน์,จันทะไชย,I,25650317,90274255,25670105,9027425525670105,90274255,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26790,90278488,000003,วันดี,ยาสาธร,I,25650429,90278488,25670628,9027848825670628,90278488,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [54]:
# รับวันที่ปัจจุบันในรูปแบบที่ต้องการ
now = datetime.now().strftime("%d%m%Y")

# กำหนดพาธของโฟลเดอร์
file_path1 = r'C:\Users\charoenchai.chu\Desktop\Rules\เคสบางสะพาน'
os.makedirs(file_path1, exist_ok=True)  # สร้างโฟลเดอร์ถ้ายังไม่มี

# สร้างชื่อไฟล์ด้วยวันที่
file_name_finish = f"เคสบางสะพาน_ALL_{now}.xlsx"
file_path_finish = os.path.join(file_path1, file_name_finish)

# ส่งออก DataFrame เป็นไฟล์ Excel
df_left_join_4.to_excel(file_path_finish, index=False)